# Second Round of Test

In the last test, we narrowed down the scope of LLM choice to Gemini 1.5 Pro and OpenAI newer models. And in the second round of test, we are going to:

(1) Update the prompts to generate both questions and annotations
(2) Test Gemini 1.5 Pro, GPT-4o, GPT-4.1, GPT-4.1-mini on the new prompts
(3) Compare the quality of the results and also consider other factors like costs, token amounts, etc.

You can run the test codes directly in this file and you need to add your own API key in a '.env' file at the same folder of this file.
- "GOOGLE_API_KEY="
- "OPENAI_API_KEY="

See the summary analysis at the end.

## Gemini Test

In [1]:
from langgraph.graph import StateGraph
from langchain_core.messages import HumanMessage
from langchain_core.runnables import Runnable
from dotenv import load_dotenv
from pydantic import BaseModel
from langchain_google_genai import ChatGoogleGenerativeAI

# Load environment variables
load_dotenv()

# Initialize LLM
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.3)

# Define the state schema
class SharedInput(BaseModel):
    input: str
    extracted_info: str = None
    questions: str = None
    prompts: str = None
    evaluation: str = None

# Agent A: Extract Information
class ExtractorAgent(Runnable):
    def invoke(self, state, config=None):
        prompt = f"Extract the main ideas and keywords from this text:\n\n{state.input}"
        response = llm.invoke([HumanMessage(content=prompt)])
        return {"extracted_info": response.content}

# Agent B: Generate Questions and Teacher Prompts
class ScaffoldPromptAgent(Runnable):
    def invoke(self, state, config=None):
        prompt = (
            "Based on the following information, do the following:\n\n"
            "1. Generate 4 comprehension questions aligned with the social, personal, cognitive, and knowledge-building dimensions of the Reading Apprenticeship (RA) framework.\n"
            "   - Social: Promotes discussion or collaborative thinking\n"
            "   - Personal: Invites self-reflection or personal connection\n"
            "   - Cognitive: Encourages metacognition or strategy use\n"
            "   - Knowledge-Building: Deepens understanding of concepts or information\n\n"
            "2. For each question, create a short teacher prompt (1–2 sentences) that guides the teacher to facilitate discussion or thinking about the question with students.\n\n"
            "Format your answer as follows:\n"
            "Questions:\n1. ...\n2. ...\n3. ...\n4. ...\n\n"
            "Prompts:\n1. ...\n2. ...\n3. ...\n4. ...\n\n"
            f"Information:\n{state.extracted_info}\n"
        )
        response = llm.invoke([HumanMessage(content=prompt)])

        # Split output into questions and prompts
        output = response.content
        try:
            questions_part = output.split("Prompts:")[0].replace("Questions:", "").strip()
            prompts_part = output.split("Prompts:")[1].strip()
        except Exception:
            questions_part = output
            prompts_part = ""
        return {
            "questions": questions_part,
            "prompts": prompts_part
        }

# Quality Check Agent
class QualityAgent(Runnable):
    def invoke(self, state, config=None):
        prompt = (
            "Evaluate the following comprehension questions and teacher prompts in terms of how well they align with the dimensions of the "
            "Reading Apprenticeship (RA) framework: Social, Personal, Cognitive, and Knowledge-Building.\n\n"
            "For each question:\n"
            "- Identify which RA dimension it most aligns with\n"
            "- Justify the alignment in 1–2 sentences\n"
            "- Evaluate whether the question is clear, relevant, and promotes meaningful thinking\n\n"
            "For each teacher prompt:\n"
            "- Evaluate if it effectively guides the teacher to facilitate student discussion or thinking\n"
            "- Check if it supports deeper understanding or useful connections\n\n"
            f"Questions:\n{state.questions}\n\nPrompts:\n{state.prompts}\n\n"
            "Return your evaluation in a structured, readable format."
        )
        response = llm.invoke([HumanMessage(content=prompt)])
        return {"evaluation": response.content}

# Create the LangGraph workflow
workflow = StateGraph(SharedInput)

# Add nodes (agents)
workflow.add_node("extractor", ExtractorAgent())
workflow.add_node("scaffold_prompt", ScaffoldPromptAgent())
workflow.add_node("quality", QualityAgent())

# Define graph edges
workflow.set_entry_point("extractor")
workflow.add_edge("extractor", "scaffold_prompt")
workflow.add_edge("scaffold_prompt", "quality")
workflow.set_finish_point("quality")

# Compile the graph
app = workflow.compile()

# Run a test
if __name__ == "__main__":
    input_text = (
        "Photosynthesis is a fundamental biological process used by green plants, algae, and some bacteria to convert "
        "light energy, usually from the sun, into chemical energy stored in glucose. During this process, organisms absorb "
        "carbon dioxide from the air and water from the soil. Using sunlight captured by chlorophyll—the green pigment in "
        "plant cells—they convert these raw materials into glucose, a type of sugar that serves as food, and release oxygen "
        "as a byproduct. This process not only sustains the plant's growth and energy needs but also produces the oxygen essential "
        "for the survival of most living organisms on Earth."
    )
    input_state = SharedInput(input=input_text)
    result = app.invoke(input_state)

    print("Step 1: Extracted Info")
    print(result.get("extracted_info", "[missing]"), "\n")

    print("Step 2: Generated Questions")
    print(result.get("questions", "[missing]"), "\n")

    print("Step 3: Generated Prompts")
    print(result.get("prompts", "[missing]"), "\n")

    print("Step 4: Quality Evaluation")
    print(result.get("evaluation", "[missing]"))


Step 1: Extracted Info
Here are the main ideas and keywords from the text:

**Main Ideas:**

*   **Definition & Purpose:** Photosynthesis is a fundamental biological process that converts light energy (from the sun) into chemical energy, stored as glucose.
*   **Participants:** It is carried out by green plants, algae, and some bacteria.
*   **Inputs:** The process requires carbon dioxide (from air) and water (from soil), with sunlight captured by chlorophyll.
*   **Outputs:** It produces glucose (which serves as food/energy for the organism) and releases oxygen as a byproduct.
*   **Significance:** Photosynthesis is vital for the plant's growth and energy, and it produces the oxygen essential for the survival of most life on Earth.

**Keywords:**

*   Photosynthesis
*   Biological process
*   Light energy
*   Chemical energy
*   Glucose
*   Green plants
*   Algae
*   Bacteria
*   Carbon dioxide (CO2)
*   Water (H2O)
*   Sunlight
*   Chlorophyll
*   Oxygen (O2)
*   Byproduct
*   Food
*

Step 1: Extracted Info
Here are the main ideas and keywords from the text:

**Main Ideas:**

*   **Definition & Purpose:** Photosynthesis is a fundamental biological process that converts light energy (from the sun) into chemical energy, stored as glucose.
*   **Participants:** It is carried out by green plants, algae, and some bacteria.
*   **Inputs:** The process requires carbon dioxide (from air) and water (from soil), with sunlight captured by chlorophyll.
*   **Outputs:** It produces glucose (which serves as food/energy for the organism) and releases oxygen as a byproduct.
*   **Significance:** Photosynthesis is vital for the plant's growth and energy, and it produces the oxygen essential for the survival of most life on Earth.

**Keywords:**

*   Photosynthesis
*   Biological process
*   Light energy
*   Chemical energy
*   Glucose
*   Green plants
*   Algae
*   Bacteria
*   Carbon dioxide (CO2)
*   Water (H2O)
*   Sunlight
*   Chlorophyll
*   Oxygen (O2)
*   Byproduct
*   Food
*   Energy
*   Survival 

Step 2: Generated Questions
1.  **Social:** If you were working with a partner to create a simple diagram illustrating photosynthesis, what are the three most crucial elements you would ensure are clearly labeled and why?
2.  **Personal:** Reflecting on your own experiences, what's one way you've observed the 'significance' of photosynthesis in the world around you, perhaps without even realizing it at the time?
3.  **Cognitive:** When you read about the inputs and outputs of photosynthesis, what mental strategy or visual did you use to keep them straight? How did that help you understand the process?
4.  **Knowledge-Building:** Based on the definition and significance provided, explain why photosynthesis is considered a 'fundamental biological process' for *all* life on Earth, not just plants. 

Step 3: Generated Prompts
1.  Pair students up and ask them to discuss their choices, justifying why those elements are essential for understanding the core process. Encourage them to listen to each other's reasoning.
2.  Give students a moment to think individually before inviting them to share. Encourage them to connect the abstract concept to a concrete observation from their own lives.
3.  Ask students to share their thinking processes. This helps normalize different learning strategies and encourages students to reflect on *how* they learn best.
4.  Challenge students to synthesize information from different parts of the text. Guide them to connect the production of oxygen and glucose to the broader ecosystem. 

Step 4: Quality Evaluation
Here's an evaluation of the comprehension questions and teacher prompts based on the Reading Apprenticeship (RA) framework:

---

### **Evaluation of Questions**

**1. Question:** If you were working with a partner to create a simple diagram illustrating photosynthesis, what are the three most crucial elements you would ensure are clearly labeled and why?

*   **RA Dimension Alignment:** **Social**
*   **Justification:** The question explicitly frames the task as "working with a partner," emphasizing collaboration and shared understanding in the learning process. It encourages students to think about how they would communicate their understanding to another person.
*   **Evaluation:**
    *   **Clarity:** The question is clear and easy to understand.
    *   **Relevance:** Highly relevant to understanding the core components of photosynthesis.
    *   **Promotes Meaningful Thinking:** Yes, it requires students to prioritize information, synthesize their understanding into a visual representation, and justify their choices, all while considering a collaborative context.

---

**2. Question:** Reflecting on your own experiences, what's one way you've observed the 'significance' of photosynthesis in the world around you, perhaps without even realizing it at the time?

*   **RA Dimension Alignment:** **Personal**
*   **Justification:** This question directly asks students to connect the academic concept of photosynthesis to their "own experiences" and observations in "the world around you," tapping into their prior knowledge and personal relevance.
*   **Evaluation:**
    *   **Clarity:** The question is clear and invites personal reflection.
    *   **Relevance:** Very relevant, as it helps students see the real-world application and impact of a scientific concept, making it less abstract.
    *   **Promotes Meaningful Thinking:** Yes, it encourages reflective thinking, observation, and the ability to link abstract scientific principles to concrete, everyday phenomena.

---

**3. Question:** When you read about the inputs and outputs of photosynthesis, what mental strategy or visual did you use to keep them straight? How did that help you understand the process?

*   **RA Dimension Alignment:** **Cognitive**
*   **Justification:** This question directly probes students' metacognitive awareness by asking about the "mental strategy or visual" they employed to comprehend and organize information. It focuses on *how* they learned and *how* that strategy aided their understanding.
*   **Evaluation:**
    *   **Clarity:** The question is clear and specific about metacognitive processes.
    *   **Relevance:** Highly relevant to developing effective reading and learning strategies.
    *   **Promotes Meaningful Thinking:** Yes, it fosters metacognition, encouraging students to reflect on their own learning processes, identify effective strategies, and become more strategic readers.

---

**4. Question:** Based on the definition and significance provided, explain why photosynthesis is considered a 'fundamental biological process' for *all* life on Earth, not just plants.

*   **RA Dimension Alignment:** **Knowledge-Building**
*   **Justification:** This question requires students to synthesize information from different parts of the text (definition, significance) and extend their understanding to explain a broader, interconnected concept (its importance for *all* life). This goes beyond recall to constructing deeper, integrated knowledge.
*   **Evaluation:**
    *   **Clarity:** The question is clear and sets a specific task for explanation.
    *   **Relevance:** Extremely relevant to grasping the overarching importance and interconnectedness of biological systems.
    *   **Promotes Meaningful Thinking:** Yes, it demands higher-order thinking skills such as synthesis, critical analysis, and the ability to articulate complex cause-and-effect relationships within a broader biological context.

---

### **Evaluation of Teacher Prompts**

**1. Prompt:** Pair students up and ask them to discuss their choices, justifying why those elements are essential for understanding the core process. Encourage them to listen to each other's reasoning.

*   **Effectively Guides Teacher:** Yes, this prompt provides clear, actionable steps for the teacher. It specifies the grouping strategy ("pair students up"), the task ("discuss their choices, justifying"), and a key social skill to encourage ("listen to each other's reasoning").
*   **Supports Deeper Understanding/Useful Connections:** Yes, by requiring students to articulate and justify their reasoning to a peer, it deepens their own understanding. Listening to others' perspectives and justifications also helps students make new connections and refine their own thinking.

---

**2. Prompt:** Give students a moment to think individually before inviting them to share. Encourage them to connect the abstract concept to a concrete observation from their own lives.

*   **Effectively Guides Teacher:** Yes, this prompt effectively guides the teacher by suggesting a valuable pre-sharing step ("moment to think individually") and providing a clear focus for the discussion ("connect the abstract concept to a concrete observation").
*   **Supports Deeper Understanding/Useful Connections:** Yes, allowing individual reflection time helps students formulate their thoughts more deeply. Encouraging the connection between abstract concepts and concrete personal experiences makes the learning more meaningful, memorable, and relevant, thereby supporting deeper understanding.

---

**3. Prompt:** Ask students to share their thinking processes. This helps normalize different learning strategies and encourages students to reflect on *how* they learn best.

*   **Effectively Guides Teacher:** Yes, this prompt clearly instructs the teacher to facilitate a discussion about "thinking processes." It also provides a valuable rationale for this activity ("helps normalize different learning strategies and encourages students to reflect on *how* they learn best"), which helps the teacher understand the pedagogical purpose.
*   **Supports Deeper Understanding/Useful Connections:** Yes, by making explicit the often-implicit cognitive strategies, students gain awareness of their own learning and are exposed to alternative approaches. This supports deeper understanding not just of the content, but of the *process* of learning, fostering metacognitive skills crucial for future learning.

---

**4. Prompt:** Challenge students to synthesize information from different parts of the text. Guide them to connect the production of oxygen and glucose to the broader ecosystem.

*   **Effectively Guides Teacher:** Yes, this prompt provides a clear instructional goal ("challenge students to synthesize information") and offers specific guidance on *what* to connect ("production of oxygen and glucose to the broader ecosystem"). This helps the teacher frame the discussion effectively.
*   **Supports Deeper Understanding/Useful Connections:** Yes, by pushing students to synthesize information and connect specific outputs to the "broader ecosystem," it directly supports the construction of a more comprehensive and interconnected understanding of the topic. This moves beyond isolated facts to a holistic view of the biological process and its significance.

In [ ]:
from langgraph.graph import StateGraph
from langchain_core.messages import HumanMessage
from langchain_core.runnables import Runnable
from dotenv import load_dotenv
from pydantic import BaseModel
from langchain_google_genai import ChatGoogleGenerativeAI


# Load environment variables
load_dotenv()

# Initialize LLM
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.3)

# Define the state schema
class SharedInput(BaseModel):
    input: str
    extracted_info: str = None
    questions: str = None
    annotations: str = None
    merged_output: str = None
    evaluation: str = None

# Agent A: Extract Information
class ExtractorAgent(Runnable):
    def invoke(self, state, config=None):
        prompt = f"Extract the main ideas and keywords from this text:\n\n{state.input}"
        response = llm.invoke([HumanMessage(content=prompt)])
        return {"extracted_info": response.content}

# Agent B: Generate Questions
class ScaffoldAgent(Runnable):
    def invoke(self, state, config=None):
        prompt = (
            "Based on the following information, generate 4 comprehension questions, aligned with social, personal, cognitive, knowledge-building dimensions of the "
            "Reading Apprenticeship framework:\n\n"
            "- **Social**: Promotes discussion or collaborative thinking\n"
            "- **Personal**: Invites self-reflection or personal connection\n"
            "- **Cognitive**: Encourages metacognition or strategy use\n"
            "- **Knowledge-Building**: Deepens understanding of concepts or information\n\n"
            f"Text:\n{state.extracted_info}\n\n"
        )
        response = llm.invoke([HumanMessage(content=prompt)])
        return {"questions": response.content}

# Agent C: Generate Annotations
class AnnotatorAgent(Runnable):
    def invoke(self, state, config=None):
        prompt = (
            "Create explanatory annotations for the following extracted information. Annotations should clarify key terms, elaborate on important ideas, "
            "and help readers connect the information to broader concepts. Use concise, accessible language suitable for learners:\n\n"
            f"{state.extracted_info}\n"
        )
        response = llm.invoke([HumanMessage(content=prompt)])
        return {"annotations": response.content}

# Merge Agent: Combine Questions and Annotations
class MergeAgent(Runnable):
    def invoke(self, state, config=None):
        merged = f"Questions:\n{state.questions}\n\nAnnotations:\n{state.annotations}"
        return {"merged_output": merged}

# Quality Check Agent
class QualityAgent(Runnable):
    def invoke(self, state, config=None):
        prompt = (
            "Evaluate the following comprehension questions and annotations in terms of how well they align with the dimensions of the "
            "Reading Apprenticeship (RA) framework: Social, Personal, Cognitive, and Knowledge-Building.\n\n"
            "For each question:\n"
            "- Identify which RA dimension it most aligns with\n"
            "- Justify the alignment in 1–2 sentences\n"
            "- Evaluate whether the question is clear, relevant, and promotes meaningful thinking\n\n"
            "For each annotation:\n"
            "- Evaluate if it clearly explains key ideas or terms\n"
            "- Check if it supports deeper understanding or useful connections\n\n"
            f"{state.merged_output}\n\n"
            "Return your evaluation in a structured, readable format."
        )
        response = llm.invoke([HumanMessage(content=prompt)])
        return {"evaluation": response.content}

# Create the LangGraph workflow
workflow = StateGraph(SharedInput)

# Add nodes (agents)
workflow.add_node("extractor", ExtractorAgent())
workflow.add_node("scaffold", ScaffoldAgent())
workflow.add_node("annotator", AnnotatorAgent())
workflow.add_node("merge", MergeAgent())
workflow.add_node("quality", QualityAgent())

# Define graph edges
workflow.set_entry_point("extractor")
workflow.add_edge("extractor", "scaffold")
workflow.add_edge("extractor", "annotator")
workflow.add_edge("scaffold", "merge")
workflow.add_edge("annotator", "merge")
workflow.add_edge("merge", "quality")
workflow.set_finish_point("quality")

# Compile the graph
app = workflow.compile()

# Run a test
if __name__ == "__main__":
    input_text = (
        "Photosynthesis is a fundamental biological process used by green plants, algae, and some bacteria to convert "
        "light energy, usually from the sun, into chemical energy stored in glucose. During this process, organisms absorb "
        "carbon dioxide from the air and water from the soil. Using sunlight captured by chlorophyll—the green pigment in "
        "plant cells—they convert these raw materials into glucose, a type of sugar that serves as food, and release oxygen "
        "as a byproduct. This process not only sustains the plant's growth and energy needs but also produces the oxygen essential "
        "for the survival of most living organisms on Earth."
    )
    input_state = SharedInput(input=input_text)
    result = app.invoke(input_state)

    # Print all intermediate outputs
    print("Step 1: Extracted Info")
    print(result.get("extracted_info", "[missing]"), "\n")

    print("Step 2: Generated Questions")
    print(result.get("questions", "[missing]"), "\n")

    print("Step 3: Generated Annotations")
    print(result.get("annotations", "[missing]"), "\n")

    print("Step 4: Quality Evaluation")
    print(result.get("evaluation", "[missing]"))


## Gemini-2.5-Flash Results

Step 1: Extracted Info
Here are the main ideas and keywords from the text:

**Main Ideas:**

*   Photosynthesis is a fundamental biological process performed by green plants, algae, and some bacteria.
*   Its primary function is to convert light energy (usually from the sun) into chemical energy, stored as glucose.
*   The process utilizes carbon dioxide and water as raw materials, with sunlight captured by chlorophyll.
*   It produces glucose (which serves as food/energy for the organism) and releases oxygen as a byproduct.
*   Photosynthesis is crucial for sustaining plant growth and providing the oxygen essential for most living organisms on Earth.

**Keywords:**

*   Photosynthesis
*   Biological process
*   Green plants
*   Algae
*   Bacteria
*   Light energy
*   Chemical energy
*   Glucose
*   Carbon dioxide
*   Water
*   Sunlight
*   Chlorophyll
*   Oxygen
*   Food
*   Energy
*   Survival
*   Living organisms 

Step 2: Generated Questions
Here are 4 comprehension questions aligned with the Reading Apprenticeship framework dimensions:

1.  **Social:** Imagine you need to explain the importance of photosynthesis to someone who doesn't understand why it matters. What key points would you emphasize to convince them of its crucial role for life on Earth, and why would you choose those specific points? (This question encourages students to articulate their understanding for an audience, promoting discussion and justification.)

2.  **Personal:** Considering that photosynthesis produces the oxygen we breathe and forms the basis for much of the food we eat, how does understanding this process personally impact your appreciation for plants or the natural environment around you? (This question invites self-reflection and encourages students to connect the scientific information to their own lives and experiences.)

3.  **Cognitive:** The text lists several keywords related to photosynthesis. If you were trying to remember the core components and products of this process, which 3-4 keywords would you focus on first, and what specific strategy would you use to link them together in your mind to ensure you understand the process? (This question prompts metacognition by asking students to think about their own learning strategies and how they would approach remembering complex information.)

4.  **Knowledge-Building:** The text states that photosynthesis converts light energy into chemical energy and is crucial for sustaining plant growth and providing oxygen. Based on this, explain the interconnectedness between the sun, plants, and most living organisms on Earth, detailing how energy and essential gases flow through this process. (This question requires students to synthesize multiple pieces of information from the text to build a deeper, more comprehensive understanding of the entire system and its implications.) 

Step 3: Generated Annotations
Here are explanatory annotations for the provided information on photosynthesis:

---

### Explanatory Annotations for Photosynthesis

**Main Ideas & Their Elaborations:**

*   **Photosynthesis is a fundamental biological process performed by green plants, algae, and some bacteria.**
    *   **Annotation:** This highlights photosynthesis as an **essential life process** (biological process) that forms the base of most food webs. It's carried out by specific organisms known as **producers** (green plants, algae, some bacteria) because they create their own food, rather than consuming other organisms.

*   **Its primary function is to convert light energy (usually from the sun) into chemical energy, stored as glucose.**
    *   **Annotation:** This describes the core transformation: **light energy** (e.g., from sunlight) is captured and changed into **chemical energy**, which is a form of energy stored within the bonds of molecules. **Glucose** (a simple sugar) is the specific molecule where this energy is stored, acting as the plant's immediate food source.

*   **The process utilizes carbon dioxide and water as raw materials, with sunlight captured by chlorophyll.**
    *   **Annotation:** This explains the "ingredients" and the energy capture mechanism. **Carbon dioxide** (a gas from the air) and **water** (from the soil) are the basic building blocks. **Sunlight** provides the energy, which is absorbed by **chlorophyll**, the green pigment found in plants, acting like a tiny solar panel.

*   **It produces glucose (which serves as food/energy for the organism) and releases oxygen as a byproduct.**
    *   **Annotation:** This details the outputs. **Glucose** is the main product, providing the plant with **food** and **energy** for growth, repair, and all life processes. **Oxygen** is a vital **byproduct** released into the atmosphere, essential for the respiration (breathing) of most living things, including humans.

*   **Photosynthesis is crucial for sustaining plant growth and providing the oxygen essential for most living organisms on Earth.**
    *   **Annotation:** This emphasizes the immense importance. It ensures the **survival** and **growth** of plants themselves and, by releasing **oxygen**, makes Earth habitable for a vast majority of **living organisms**, forming the foundation of nearly all ecosystems and food chains.

---

**Keywords & Their Clarifications:**

*   **Photosynthesis:** From Greek words "photo" (light) and "synthesis" (to make). It literally means "making with light," referring to how plants make their food.
*   **Biological process:** A series of steps or reactions that occur naturally within living organisms to maintain life.
*   **Green plants, Algae, Bacteria:** These are the primary types of organisms capable of photosynthesis. They are often called "autotrophs" (self-feeders) or "producers" because they create their own food.
*   **Light energy:** The energy carried by light waves, typically from the sun, which powers the photosynthetic reaction.
*   **Chemical energy:** Energy stored within the bonds of chemical compounds (like glucose). This energy can be released when these bonds are broken, providing fuel for life processes.
*   **Glucose:** A simple sugar molecule (C6H12O6) that is the primary form of chemical energy produced and stored by plants. It's the plant's direct "food" and energy source.
*   **Carbon dioxide:** A gas (CO2) present in the atmosphere that plants absorb through tiny pores in their leaves and use as a raw material to build glucose.
*   **Water:** A liquid (H2O) absorbed by plants from the soil through their roots, serving as another essential raw material for photosynthesis.
*   **Sunlight:** The primary natural source of light energy for photosynthesis on Earth, driving the entire process.
*   **Chlorophyll:** The green pigment found in plant cells (specifically in chloroplasts) that is responsible for absorbing sunlight and initiating the photosynthetic process.
*   **Oxygen:** A gas (O2) released into the atmosphere as a byproduct of photosynthesis. It is vital for the respiration (energy release) of most living things, including animals and humans.
*   **Food:** In the context of photosynthesis, "food" refers to the glucose produced, which provides energy and building blocks for the plant's growth and survival.
*   **Energy:** The capacity to do work or cause change. In photosynthesis, light energy is converted into chemical energy.
*   **Survival:** The ability of an organism or species to continue living. Photosynthesis is fundamental to the survival of plants and, indirectly, to the survival of most life on Earth.
*   **Living organisms:** All forms of life, from microscopic bacteria to large animals and plants, many of which depend on photosynthesis for their food (directly or indirectly) and the oxygen they breathe. 

Step 4: Quality Evaluation
Here's an evaluation of the comprehension questions and annotations based on the Reading Apprenticeship (RA) framework:

---

### Evaluation of Comprehension Questions

**1. Question: Social**
*   **Question:** Imagine you need to explain the importance of photosynthesis to someone who doesn't understand why it matters. What key points would you emphasize to convince them of its crucial role for life on Earth, and why would you choose those specific points?
*   **Identified RA Dimension:** Social
*   **Justification:** This question explicitly requires students to articulate their understanding for an external audience, simulating a real-world communication scenario. It promotes the social aspect of learning by asking them to consider how to effectively convey information and justify their choices, which could lead to discussion and shared understanding.
*   **Evaluation:**
    *   **Clear:** Yes, the scenario and task are clearly defined.
    *   **Relevant:** Highly relevant, as it assesses not just comprehension but the ability to communicate that understanding effectively, a key skill in any field.
    *   **Promotes meaningful thinking:** Yes, it moves beyond simple recall to require synthesis, prioritization of information, and strategic thinking about audience and persuasion.

**2. Question: Personal**
*   **Question:** Considering that photosynthesis produces the oxygen we breathe and forms the basis for much of the food we eat, how does understanding this process personally impact your appreciation for plants or the natural environment around you?
*   **Identified RA Dimension:** Personal
*   **Justification:** This question directly prompts students to connect the scientific concept to their own lives, experiences, and feelings ("personally impact your appreciation"). It encourages self-reflection and helps students build a personal stake in the material, making the learning more meaningful and memorable.
*   **Evaluation:**
    *   **Clear:** Yes, the question is straightforward and asks for a personal reflection.
    *   **Relevant:** Very relevant, as personal connection enhances engagement and long-term retention of knowledge. It also fosters a sense of wonder and responsibility towards the environment.
    *   **Promotes meaningful thinking:** Yes, it encourages students to move beyond factual recall to consider the broader implications of the scientific process on their own lives and perspectives.

**3. Question: Cognitive**
*   **Question:** The text lists several keywords related to photosynthesis. If you were trying to remember the core components and products of this process, which 3-4 keywords would you focus on first, and what specific strategy would you use to link them together in your mind to ensure you understand the process?
*   **Identified RA Dimension:** Cognitive
*   **Justification:** This question is a strong example of promoting metacognition, a core aspect of the Cognitive dimension. It asks students to think explicitly about their own learning processes, identify key information, and articulate specific strategies they would employ for comprehension and retention.
*   **Evaluation:**
    *   **Clear:** Yes, the task of selecting keywords and describing a strategy is clear.
    *   **Relevant:** Highly relevant, as it directly addresses effective study habits and comprehension monitoring. It helps students become more aware and intentional learners.
    *   **Promotes meaningful thinking:** Yes, it encourages strategic thinking about learning, self-assessment of understanding, and the development of effective cognitive strategies.

**4. Question: Knowledge-Building**
*   **Question:** The text states that photosynthesis converts light energy into chemical energy and is crucial for sustaining plant growth and providing oxygen. Based on this, explain the interconnectedness between the sun, plants, and most living organisms on Earth, detailing how energy and essential gases flow through this process.
*   **Identified RA Dimension:** Knowledge-Building
*   **Justification:** This question requires students to synthesize multiple pieces of information from the text (and potentially prior knowledge) to construct a more complex and holistic understanding of a system. It asks them to explain relationships and flows, thereby building a deeper, interconnected knowledge structure.
*   **Evaluation:**
    *   **Clear:** Yes, the prompt clearly asks for an explanation of interconnectedness and flow.
    *   **Relevant:** Extremely relevant, as it assesses a student's ability to move beyond isolated facts to grasp the larger conceptual framework and implications of photosynthesis within an ecosystem.
    *   **Promotes meaningful thinking:** Yes, it demands high-level thinking, including analysis, synthesis, and the ability to articulate complex causal relationships.

---

### Evaluation of Annotations

**Overall Impression of Annotations:** The annotations are well-structured, clear, and consistently add value beyond simply restating the text. They effectively explain key ideas, clarify terms, and support deeper understanding by providing context, implications, and connections.

**Main Ideas & Their Elaborations Annotations:**

*   **"Photosynthesis is a fundamental biological process..."**
    *   **Clarity/Explanation:** Clearly explains "essential life process" and introduces "producers" effectively.
    *   **Deeper Understanding/Connections:** Yes, by linking it to "base of most food webs" and defining "producers," it immediately places photosynthesis within a broader ecological context.

*   **"Its primary function is to convert light energy..."**
    *   **Clarity/Explanation:** Clearly defines "chemical energy" and specifies "glucose" as the storage molecule and "immediate food source."
    *   **Deeper Understanding/Connections:** Yes, it clarifies the *what* and *why* of the energy conversion, making the abstract concept of energy more concrete.

*   **"The process utilizes carbon dioxide and water..."**
    *   **Clarity/Explanation:** Clearly identifies "ingredients" and explains chlorophyll's role as a "tiny solar panel," a very effective analogy.
    *   **Deeper Understanding/Connections:** Yes, the analogy helps visualize the mechanism, and identifying raw materials is crucial for understanding the process.

*   **"It produces glucose... and releases oxygen..."**
    *   **Clarity/Explanation:** Clearly distinguishes between the main product (glucose for food/energy) and the vital byproduct (oxygen).
    *   **Deeper Understanding/Connections:** Yes, it emphasizes the dual importance of the outputs for the plant itself and for other living things, reinforcing the interconnectedness.

*   **"Photosynthesis is crucial for sustaining plant growth..."**
    *   **Clarity/Explanation:** Clearly reiterates and expands on the immense importance.
    *   **Deeper Understanding/Connections:** Yes, by linking it to "survival," "habitable Earth," "ecosystems," and "food chains," it provides a comprehensive understanding of the process's global significance.

**Keywords & Their Clarifications Annotations:**

*   **Photosynthesis:**
    *   **Clarity/Explanation:** Excellent, provides etymology ("making with light") which aids memorization and understanding.
    *   **Deeper Understanding/Connections:** Yes, the etymology is a useful mnemonic and conceptual anchor.

*   **Biological process:**
    *   **Clarity/Explanation:** Clear and concise definition.
    *   **Deeper Understanding/Connections:** Yes, provides a foundational definition for understanding the context of photosynthesis.

*   **Green plants, Algae, Bacteria:**
    *   **Clarity/Explanation:** Clearly identifies them and introduces "autotrophs" and "producers."
    *   **Deeper Understanding/Connections:** Yes, these additional terms are crucial for building a more sophisticated scientific vocabulary and understanding ecological roles.

*   **Light energy:**
    *   **Clarity/Explanation:** Clear definition and source.
    *   **Deeper Understanding/Connections:** Yes, establishes the energy input.

*   **Chemical energy:**
    *   **Clarity/Explanation:** Clearly explains energy stored in bonds and its release for fuel.
    *   **Deeper Understanding/Connections:** Yes, crucial for understanding the transformation aspect of photosynthesis.

*   **Glucose:**
    *   **Clarity/Explanation:** Provides chemical formula, identifies it as a simple sugar, and the primary form of energy.
    *   **Deeper Understanding/Connections:** Yes, gives specific detail and reinforces its role as the plant's "food."

*   **Carbon dioxide, Water:**
    *   **Clarity/Explanation:** Clear identification of source and role as raw materials.
    *   **Deeper Understanding/Connections:** Yes, essential for understanding the inputs.

*   **Sunlight:**
    *   **Clarity/Explanation:** Clear identification as the primary energy source.
    *   **Deeper Understanding/Connections:** Yes, reinforces its fundamental role.

*   **Chlorophyll:**
    *   **Clarity/Explanation:** Clearly identifies it as the green pigment and its function in absorbing sunlight.
    *   **Deeper Understanding/Connections:** Yes, explains the mechanism of light capture.

*   **Oxygen:**
    *   **Clarity/Explanation:** Clearly identifies it as a byproduct and its vital role in respiration.
    *   **Deeper Understanding/Connections:** Yes, highlights its importance for other life forms.

*   **Food, Energy, Survival, Living organisms:**
    *   **Clarity/Explanation:** All are clearly defined within the context of photosynthesis.
    *   **Deeper Understanding/Connections:** These terms provide essential context and reinforce the broader implications and significance of the process. For example, "Survival" explicitly links photosynthesis to the continuation of life.

**Conclusion for Annotations:** The annotations are exceptionally well-done. They are concise, precise, and consistently add value by clarifying terms, providing context, making connections, and highlighting the significance of the information. They effectively support deeper understanding and would be very useful for a learner.

## Gemini 1.5 Pro results

Step 1: Extracted Info
**Main Ideas:**

* Photosynthesis converts light energy into chemical energy (glucose).
* Plants, algae, and some bacteria perform photosynthesis.
* Photosynthesis consumes carbon dioxide and water, and produces glucose and oxygen.
* Photosynthesis is essential for plant life and provides oxygen for most other life on Earth.


**Keywords:**

* Photosynthesis
* Light energy
* Chemical energy
* Glucose
* Plants
* Algae
* Bacteria
* Carbon dioxide
* Water
* Sunlight
* Chlorophyll
* Oxygen 

Step 2: Generated Questions
1. **Social:** Imagine you're explaining photosynthesis to a younger sibling. How would you describe the process using everyday language, and what analogy could you use to help them understand the transformation of light energy into chemical energy?

2. **Personal:**  Think about the air you breathe. How does understanding the process of photosynthesis affect your appreciation for plants and other photosynthetic organisms in your environment?

3. **Cognitive:** What strategies did you use to understand the relationship between the keywords listed (e.g., sunlight, chlorophyll, carbon dioxide) and the overall process of photosynthesis?  Which keyword was most helpful in anchoring your understanding, and why?

4. **Knowledge-Building:**  Beyond its role in producing oxygen, how does photosynthesis contribute to the Earth's ecosystems and the food web?  Consider both direct and indirect impacts. 

Step 3: Generated Annotations
**Main Ideas:**

* **Photosynthesis converts light energy into chemical energy (glucose).**  *Annotation:* This is the core concept of photosynthesis.  Think of it like a solar panel: it takes sunlight (light energy) and transforms it into usable energy (chemical energy) stored in sugar (glucose). This stored energy fuels the plant's activities, like growing and reproducing.
* **Plants, algae, and some bacteria perform photosynthesis.** *Annotation:* These organisms are called "photoautotrophs," meaning they can make their own food using light. Plants are the most familiar example, but microscopic algae in the ocean and certain bacteria also contribute significantly to global photosynthesis.
* **Photosynthesis consumes carbon dioxide and water, and produces glucose and oxygen.** *Annotation:* This describes the inputs and outputs of photosynthesis.  Plants take in carbon dioxide from the air and water from the soil. Using sunlight, they transform these into glucose (food) and oxygen, which is released into the atmosphere.
* **Photosynthesis is essential for plant life and provides oxygen for most other life on Earth.** *Annotation:* This highlights the global importance of photosynthesis.  It's how plants get their food, and it's the primary source of the oxygen we breathe.  Without photosynthesis, life as we know it wouldn't exist.


**Keywords:**

* **Photosynthesis:** The process by which light energy is converted into chemical energy in the form of glucose.
* **Light energy:** Energy from the sun that travels in waves.
* **Chemical energy:** Energy stored in the bonds of chemical compounds, like glucose. This energy can be released and used by organisms.
* **Glucose:** A simple sugar (C6H12O6) that serves as a primary source of energy for living organisms.  Think of it as plant food.
* **Plants:** Multicellular organisms that typically have roots, stems, and leaves and perform photosynthesis.
* **Algae:**  A diverse group of photosynthetic organisms, ranging from single-celled microalgae to large seaweeds.
* **Bacteria:** Single-celled microorganisms. Some bacteria, called cyanobacteria, are photosynthetic.
* **Carbon dioxide (CO2):** A gas in the atmosphere that is taken in by plants during photosynthesis.  It's also a byproduct of respiration in animals.
* **Water (H2O):** Essential for all life. In photosynthesis, water provides the hydrogen needed to make glucose.
* **Sunlight:** The source of light energy that drives photosynthesis.
* **Chlorophyll:** The green pigment in plants that captures light energy from the sun. This is what makes plants look green.
* **Oxygen (O2):** A gas produced during photosynthesis that is essential for the respiration of most living organisms. 

Step 4: Quality Evaluation
## Evaluation of Reading Apprenticeship Questions and Annotations

Here's an evaluation of the provided questions and annotations based on the Reading Apprenticeship framework:

**Questions:**

| Question # | RA Dimension | Justification | Clarity/Relevance/Meaningful Thinking |
|---|---|---|---|
| 1 | Social |  Focuses on explaining a concept to another person, emphasizing communication and perspective-taking. | Clear, relevant, and promotes meaningful thinking by encouraging the use of analogies and everyday language. |
| 2 | Personal | Connects the scientific concept of photosynthesis to the individual's personal experience and environment, fostering reflection and connection. | Clear, relevant, and promotes meaningful thinking by encouraging personal reflection on the importance of photosynthesis. |
| 3 | Cognitive |  Explores the learner's metacognitive strategies for understanding complex information and identifying key concepts. | Clear, relevant, and promotes meaningful thinking by prompting reflection on learning strategies and the importance of specific keywords. |
| 4 | Knowledge-Building |  Extends understanding beyond the basic facts of photosynthesis to its broader impact on ecosystems and the food web, connecting to prior knowledge and building new connections. | Clear, relevant, and promotes meaningful thinking by encouraging exploration of the wider implications of photosynthesis.


**Annotations - Main Ideas:**

| Main Idea | Clarity/Deeper Understanding/Useful Connections |
|---|---|
| Photosynthesis converts light energy into chemical energy (glucose). | Clear explanation with a helpful analogy (solar panel) that connects the abstract concept to a familiar object.  Supports deeper understanding by explaining the purpose of the stored energy. |
| Plants, algae, and some bacteria perform photosynthesis. | Clear explanation with useful terminology (photoautotrophs) and examples beyond just plants.  Broadens understanding by including algae and bacteria. |
| Photosynthesis consumes carbon dioxide and water, and produces glucose and oxygen. | Clear explanation of the inputs and outputs. Supports deeper understanding by specifying where each component comes from and goes to. |
| Photosynthesis is essential for plant life and provides oxygen for most other life on Earth. | Clear explanation of the importance of photosynthesis.  Supports deeper understanding by highlighting its global impact and connection to all life. |


**Annotations - Keywords:**

| Keyword | Clarity/Deeper Understanding/Useful Connections |
|---|---|
| Photosynthesis | Clear and concise definition. |
| Light energy | Clear definition. |
| Chemical energy | Clear definition with an explanation of its usability by organisms. |
| Glucose | Clear definition with a helpful analogy ("plant food") and chemical formula. |
| Plants | Clear definition. |
| Algae | Clear definition with a note about its diversity. |
| Bacteria | Clear definition with a mention of photosynthetic cyanobacteria. |
| Carbon dioxide (CO2) | Clear definition with connection to both plant and animal processes. |
| Water (H2O) | Clear definition with its role in glucose formation. |
| Sunlight | Clear definition. |
| Chlorophyll | Clear definition explaining its function and why plants appear green. |
| Oxygen (O2) | Clear definition and its importance for respiration. |


Overall, the questions and annotations are well-aligned with the Reading Apprenticeship framework. They are clear, relevant, and promote meaningful thinking about photosynthesis, encouraging connections between the concept and personal experiences, broader knowledge, and metacognitive strategies. The annotations effectively explain key ideas and terms, supporting deeper understanding and making useful connections to familiar concepts and real-world applications.

## ChatGPT Test

In [ ]:
from langgraph.graph import StateGraph
from langchain.schema import SystemMessage
from langchain_core.runnables import Runnable
from dotenv import load_dotenv
from pydantic import BaseModel
from langchain_openai import ChatOpenAI

# Load environment variables
load_dotenv()

# Initialize LLM (GPT-4.1-mini)
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.3)

# Define the state schema
class SharedInput(BaseModel):
    input: str
    extracted_info: str = None
    questions: str = None
    annotations: str = None
    merged_output: str = None
    evaluation: str = None

# Agent A: Extract Information
class ExtractorAgent(Runnable):
    def invoke(self, state, config=None):
        prompt = f"Extract the main ideas and keywords from this text:\n\n{state.input}"
        response = llm.invoke([SystemMessage(content=prompt)])
        return {"extracted_info": response.content}

# Agent B: Generate Questions
class ScaffoldAgent(Runnable):
    def invoke(self, state, config=None):
        prompt = (
            "Based on the following information, generate 4 comprehension questions, aligned with social, personal, cognitive, knowledge-building dimensions of the "
            "Reading Apprenticeship framework:\n\n"
            "- **Social**: Promotes discussion or collaborative thinking\n"
            "- **Personal**: Invites self-reflection or personal connection\n"
            "- **Cognitive**: Encourages metacognition or strategy use\n"
            "- **Knowledge-Building**: Deepens understanding of concepts or information\n\n"
            f"Text:\n{state.extracted_info}\n\n"
        )
        response = llm.invoke([SystemMessage(content=prompt)])
        return {"questions": response.content}

# Agent C: Generate Annotations
class AnnotatorAgent(Runnable):
    def invoke(self, state, config=None):
        prompt = (
            "Create explanatory annotations for the following extracted information. Annotations should clarify key terms, elaborate on important ideas, "
            "and help readers connect the information to broader concepts. Use concise, accessible language suitable for learners:\n\n"
            f"{state.extracted_info}\n"
        )
        response = llm.invoke([SystemMessage(content=prompt)])
        return {"annotations": response.content}

# Merge Agent: Combine Questions and Annotations
class MergeAgent(Runnable):
    def invoke(self, state, config=None):
        merged = f"Questions:\n{state.questions}\n\nAnnotations:\n{state.annotations}"
        return {"merged_output": merged}

# Quality Check Agent
class QualityAgent(Runnable):
    def invoke(self, state, config=None):
        prompt = (
            "Evaluate the following comprehension questions and annotations in terms of how well they align with the dimensions of the "
            "Reading Apprenticeship (RA) framework: Social, Personal, Cognitive, and Knowledge-Building.\n\n"
            "For each question:\n"
            "- Identify which RA dimension it most aligns with\n"
            "- Justify the alignment in 1–2 sentences\n"
            "- Evaluate whether the question is clear, relevant, and promotes meaningful thinking\n\n"
            "For each annotation:\n"
            "- Evaluate if it clearly explains key ideas or terms\n"
            "- Check if it supports deeper understanding or useful connections\n\n"
            f"{state.merged_output}\n\n"
            "Return your evaluation in a structured, readable format."
        )
        response = llm.invoke([SystemMessage(content=prompt)])
        return {"evaluation": response.content}

# Create the LangGraph workflow
workflow = StateGraph(SharedInput)

# Add nodes (agents)
workflow.add_node("extractor", ExtractorAgent())
workflow.add_node("scaffold", ScaffoldAgent())
workflow.add_node("annotator", AnnotatorAgent())
workflow.add_node("merge", MergeAgent())
workflow.add_node("quality", QualityAgent())

# Define graph edges
workflow.set_entry_point("extractor")
workflow.add_edge("extractor", "scaffold")
workflow.add_edge("extractor", "annotator")
workflow.add_edge("scaffold", "merge")
workflow.add_edge("annotator", "merge")
workflow.add_edge("merge", "quality")
workflow.set_finish_point("quality")

# Compile the graph
app = workflow.compile()

# Run a test
if __name__ == "__main__":
    input_text = (
        "Photosynthesis is a fundamental biological process used by green plants, algae, and some bacteria to convert "
        "light energy, usually from the sun, into chemical energy stored in glucose. During this process, organisms absorb "
        "carbon dioxide from the air and water from the soil. Using sunlight captured by chlorophyll—the green pigment in "
        "plant cells—they convert these raw materials into glucose, a type of sugar that serves as food, and release oxygen "
        "as a byproduct. This process not only sustains the plant's growth and energy needs but also produces the oxygen essential "
        "for the survival of most living organisms on Earth."
    )
    input_state = SharedInput(input=input_text)
    result = app.invoke(input_state)

    # Print all intermediate outputs
    print("Step 1: Extracted Info")
    print(result.get("extracted_info", "[missing]"), "\n")

    print("Step 2: Generated Questions")
    print(result.get("questions", "[missing]"), "\n")

    print("Step 3: Generated Annotations")
    print(result.get("annotations", "[missing]"), "\n")

    print("Step 4: Quality Evaluation")
    print(result.get("evaluation", "[missing]"))


### ChatGPT-4.1 Result

Step 1: Extracted Info
**Main Ideas:**
- Photosynthesis is a fundamental biological process in green plants, algae, and some bacteria.
- It converts light energy (usually from the sun) into chemical energy stored in glucose.
- Organisms absorb carbon dioxide (CO₂) and water (H₂O) as raw materials.
- Sunlight, captured by chlorophyll, drives the conversion of these materials into glucose and oxygen.
- Glucose serves as food/energy for the organism.
- Oxygen is released as a byproduct, essential for most living organisms.

**Keywords:**
- Photosynthesis
- Green plants
- Algae
- Bacteria
- Light energy
- Sunlight
- Chemical energy
- Glucose
- Carbon dioxide
- Water
- Chlorophyll
- Oxygen
- Byproduct
- Growth
- Energy needs 

Step 2: Generated Questions
**Social:**  
How might you explain the process of photosynthesis to a classmate who is struggling to understand it? What examples or comparisons could you use to help them?

**Personal:**  
Can you think of a time when you relied on plants—directly or indirectly—for energy or food? How does knowing about photosynthesis change the way you think about that experience?

**Cognitive:**  
What strategies did you use to remember the steps and key terms involved in photosynthesis? Were there any parts you found confusing, and how did you work through them?

**Knowledge-Building:**  
Why is photosynthesis considered essential for most living organisms on Earth? Explain how the process supports both plants and animals. 

Step 3: Generated Annotations
**Annotations:**

- **Photosynthesis**: This is the process by which certain organisms use sunlight to make their own food. It is essential for life on Earth because it provides the base of most food chains and produces oxygen.

- **Green plants**: These are plants that contain chlorophyll, the pigment that gives them their green color and allows them to perform photosynthesis.

- **Algae**: Simple, plant-like organisms found mostly in water. Like green plants, they use photosynthesis to make food and produce oxygen.

- **Bacteria**: Some bacteria, such as cyanobacteria (also called blue-green algae), can also perform photosynthesis, contributing to oxygen production and food chains, especially in aquatic environments.

- **Light energy**: Energy that comes from light, most often from the sun. This energy is necessary to power the chemical reactions of photosynthesis.

- **Sunlight**: The natural light from the sun, which is the primary source of energy for photosynthesis on Earth.

- **Chemical energy**: Energy stored in the bonds of chemical compounds, such as glucose. Photosynthesis transforms light energy into chemical energy, which organisms can use for growth and maintenance.

- **Glucose**: A simple sugar that serves as a primary energy source for cells. Plants make glucose during photosynthesis and use it for energy or to build other important molecules.

- **Carbon dioxide (CO₂)**: A gas present in the atmosphere. Plants, algae, and some bacteria absorb carbon dioxide from the air or water to use as a raw material in photosynthesis.

- **Water (H₂O)**: Another raw material for photosynthesis, absorbed by plants through their roots. Water provides hydrogen atoms needed to make glucose.

- **Chlorophyll**: The green pigment found in chloroplasts of plant cells and in some bacteria and algae. Chlorophyll captures sunlight and starts the process of converting it into chemical energy.

- **Oxygen (O₂)**: A gas released as a byproduct of photosynthesis. This oxygen is essential for the survival of most living organisms, which use it for respiration.

- **Byproduct**: A secondary product made during a process. In photosynthesis, oxygen is a byproduct—produced alongside glucose, but not the main goal of the process.

- **Growth**: The process by which organisms increase in size or number. Photosynthesis provides the energy and materials needed for plants, algae, and some bacteria to grow.

- **Energy needs**: All living things require energy to carry out life processes. Photosynthesis is how green plants, algae, and some bacteria meet their energy needs, and it also supports the energy needs of animals that eat these organisms.

**Connecting to Broader Concepts:**
Photosynthesis is crucial for life on Earth. It not only provides food and energy for plants and other photosynthetic organisms but also supports animals (including humans) that depend on plants for food. The oxygen produced during photosynthesis is vital for respiration in most living things, making this process central to the planet’s ecosystems and the balance of gases in the atmosphere. 

Step 4: Quality Evaluation
Certainly! Below is a structured evaluation of the comprehension questions and annotations according to the Reading Apprenticeship (RA) framework dimensions: Social, Personal, Cognitive, and Knowledge-Building.

---

## Comprehension Questions

### 1. **Social**
**Question:**  
*How might you explain the process of photosynthesis to a classmate who is struggling to understand it? What examples or comparisons could you use to help them?*

- **RA Dimension Alignment:** Social
- **Justification:** This question directly engages students in thinking about how to communicate and collaborate with peers, fostering social interaction and shared understanding.
- **Evaluation:**  
  - **Clarity:** The question is clear and easy to understand.
  - **Relevance:** Highly relevant, as peer explanation is a common and effective learning strategy.
  - **Promotes Meaningful Thinking:** Yes, it encourages students to rephrase concepts and use analogies, deepening their own understanding while considering others’ perspectives.

---

### 2. **Personal**
**Question:**  
*Can you think of a time when you relied on plants—directly or indirectly—for energy or food? How does knowing about photosynthesis change the way you think about that experience?*

- **RA Dimension Alignment:** Personal
- **Justification:** This question asks students to connect the concept of photosynthesis to their own lives and experiences, fostering personal relevance and reflection.
- **Evaluation:**  
  - **Clarity:** Clear and straightforward.
  - **Relevance:** Highly relevant, as it personalizes the scientific concept.
  - **Promotes Meaningful Thinking:** Yes, it prompts students to reflect on their own lives and reconsider everyday experiences in light of new knowledge.

---

### 3. **Cognitive**
**Question:**  
*What strategies did you use to remember the steps and key terms involved in photosynthesis? Were there any parts you found confusing, and how did you work through them?*

- **RA Dimension Alignment:** Cognitive
- **Justification:** This question focuses on metacognition, asking students to reflect on their own learning processes and problem-solving strategies.
- **Evaluation:**  
  - **Clarity:** Clear and specific.
  - **Relevance:** Relevant, as it encourages self-awareness about learning.
  - **Promotes Meaningful Thinking:** Yes, it fosters metacognitive skills and helps students become more effective learners.

---

### 4. **Knowledge-Building**
**Question:**  
*Why is photosynthesis considered essential for most living organisms on Earth? Explain how the process supports both plants and animals.*

- **RA Dimension Alignment:** Knowledge-Building
- **Justification:** This question requires students to synthesize information and explain the broader significance of photosynthesis, building conceptual understanding.
- **Evaluation:**  
  - **Clarity:** Clear and direct.
  - **Relevance:** Highly relevant, as it targets the core purpose and impact of photosynthesis.
  - **Promotes Meaningful Thinking:** Yes, it encourages students to connect facts to larger biological systems and ecological relationships.

---

## Annotations

For each annotation, I will evaluate:
- If it clearly explains key ideas or terms
- If it supports deeper understanding or useful connections

---

### **Photosynthesis**
- **Explanation:** Clearly defines the process and its importance for food chains and oxygen.
- **Support for Understanding:** Yes, it highlights both the function and significance of photosynthesis.

### **Green plants**
- **Explanation:** Clearly states what green plants are and their role in photosynthesis.
- **Support for Understanding:** Yes, connects the concept of chlorophyll to photosynthesis.

### **Algae**
- **Explanation:** Defines algae and their similarity to green plants in photosynthesis.
- **Support for Understanding:** Yes, clarifies the role of algae in food chains and oxygen production.

### **Bacteria**
- **Explanation:** Specifies which bacteria perform photosynthesis and their ecological role.
- **Support for Understanding:** Yes, expands understanding beyond plants to include bacteria.

### **Light energy**
- **Explanation:** Defines light energy and its necessity for photosynthesis.
- **Support for Understanding:** Yes, connects energy source to the process.

### **Sunlight**
- **Explanation:** Defines sunlight and its role in photosynthesis.
- **Support for Understanding:** Yes, reinforces the importance of the sun as the primary energy source.

### **Chemical energy**
- **Explanation:** Defines chemical energy and its transformation during photosynthesis.
- **Support for Understanding:** Yes, explains the conversion of energy forms.

### **Glucose**
- **Explanation:** Defines glucose and its role as an energy source.
- **Support for Understanding:** Yes, connects the product of photosynthesis to cellular energy.

### **Carbon dioxide (CO₂)**
- **Explanation:** Defines CO₂ and its role as a raw material.
- **Support for Understanding:** Yes, clarifies the input needed for photosynthesis.

### **Water (H₂O)**
- **Explanation:** Defines water’s role in photosynthesis.
- **Support for Understanding:** Yes, explains its function as a raw material.

### **Chlorophyll**
- **Explanation:** Defines chlorophyll and its function in capturing sunlight.
- **Support for Understanding:** Yes, connects pigment to energy capture.

### **Oxygen (O₂)**
- **Explanation:** Defines oxygen as a byproduct and its importance.
- **Support for Understanding:** Yes, links photosynthesis to respiration and life support.

### **Byproduct**
- **Explanation:** Defines byproduct in the context of photosynthesis.
- **Support for Understanding:** Yes, clarifies the concept of secondary products.

### **Growth**
- **Explanation:** Defines growth and its dependence on photosynthesis.
- **Support for Understanding:** Yes, connects energy and materials to organismal growth.

### **Energy needs**
- **Explanation:** Defines energy needs and how photosynthesis meets them.
- **Support for Understanding:** Yes, links photosynthesis to the survival of various organisms.

---

### **Connecting to Broader Concepts**
- **Explanation:** Clearly summarizes the global importance of photosynthesis.
- **Support for Understanding:** Yes, effectively connects the process to ecological and atmospheric balance, reinforcing its centrality to life on Earth.

---

## **Summary Table**

| Item | RA Dimension | Justification | Clarity | Relevance | Promotes Meaningful Thinking / Supports Deeper Understanding |
|------|--------------|--------------|---------|-----------|-------------------------------------------------------------|
| Social Q | Social | Peer explanation | Clear | Relevant | Yes |
| Personal Q | Personal | Connects to personal experience | Clear | Relevant | Yes |
| Cognitive Q | Cognitive | Metacognitive reflection | Clear | Relevant | Yes |
| Knowledge-Building Q | Knowledge-Building | Synthesizes and explains significance | Clear | Relevant | Yes |
| Annotations | N/A | Define and explain key terms | Clear | Relevant | Yes, all support deeper understanding |
| Connecting to Broader Concepts | N/A | Synthesizes and contextualizes | Clear | Relevant | Yes, strong connections |

---

## **Overall Evaluation**

- **Questions:** All questions are well-aligned with their respective RA dimensions, are clear, relevant, and promote meaningful thinking.
- **Annotations:** All annotations clearly explain key ideas or terms and support deeper understanding and useful connections, both within the topic and to broader concepts.

**Conclusion:**  
The comprehension questions and annotations are exemplary in supporting the Reading Apprenticeship framework. They foster social interaction, personal connection, cognitive reflection, and knowledge-building, while the annotations provide clear, accessible explanations that deepen understanding and connect to larger scientific and ecological ideas.

### ChatGPT-4.1-mini Result

Step 1: Extracted Info
Main ideas:
- Photosynthesis is a biological process in green plants, algae, and some bacteria.
- It converts light energy into chemical energy stored in glucose.
- Organisms absorb carbon dioxide and water.
- Chlorophyll captures sunlight to drive the conversion.
- Glucose is produced as food, and oxygen is released as a byproduct.
- Photosynthesis supports plant growth and provides oxygen necessary for most life on Earth.

Keywords:
Photosynthesis, green plants, algae, bacteria, light energy, chemical energy, glucose, carbon dioxide, water, chlorophyll, sunlight, oxygen, biological process, energy conversion, plant growth. 

Step 2: Generated Questions
**Social:** How might discussing the process of photosynthesis with a group help you better understand how plants contribute to the oxygen we breathe?

**Personal:** Can you think of a time when learning about photosynthesis changed how you view plants or your environment? How did it affect your appreciation for nature?

**Cognitive:** What strategies can you use to remember the steps of photosynthesis and the role of chlorophyll in converting light energy into chemical energy?

**Knowledge-Building:** How does the process of photosynthesis connect to the survival of other living organisms beyond plants, and why is it essential for life on Earth? 

Step 3: Generated Annotations
- Photosynthesis: A natural process by which certain organisms convert light energy into chemical energy, enabling them to produce food.
- Green plants, algae, bacteria: Types of organisms capable of photosynthesis; green plants and algae contain chlorophyll, while some bacteria have similar pigments.
- Light energy: Energy from sunlight that powers the photosynthesis process.
- Chemical energy: Energy stored in chemical bonds, such as those in glucose molecules produced during photosynthesis.
- Glucose: A simple sugar that serves as an energy source and building block for growth in plants and other photosynthetic organisms.
- Carbon dioxide: A gas absorbed from the air, used as a carbon source to build glucose molecules.
- Water: Absorbed by roots or cell membranes, water provides electrons and hydrogen atoms needed for glucose synthesis.
- Chlorophyll: A green pigment in chloroplasts that captures sunlight and initiates the energy conversion.
- Sunlight: The natural light from the sun, essential for driving photosynthesis.
- Oxygen: A byproduct released into the atmosphere during photosynthesis, vital for respiration in most living organisms.
- Biological process: A series of chemical reactions occurring within living organisms to sustain life.
- Energy conversion: The transformation of energy from one form (light) to another (chemical) during photosynthesis.
- Plant growth: The increase in plant size and biomass supported by the glucose produced through photosynthesis.
- Overall, photosynthesis is fundamental for life on Earth as it produces food and oxygen, supporting ecosystems and the planet’s energy flow. 

Step 4: Quality Evaluation
**Evaluation of Comprehension Questions**

| Question (Dimension) | Alignment with RA Dimension | Justification | Clarity, Relevance, and Depth |
|---------------------|-----------------------------|---------------|------------------------------|
| **Social:** How might discussing the process of photosynthesis with a group help you better understand how plants contribute to the oxygen we breathe? | Social | This question explicitly invites collaborative learning and peer discussion, aligning well with the Social dimension that emphasizes interaction and shared meaning-making. | Clear and relevant; it encourages students to reflect on the value of social interaction in learning, promoting metacognitive awareness and meaningful engagement. |
| **Personal:** Can you think of a time when learning about photosynthesis changed how you view plants or your environment? How did it affect your appreciation for nature? | Personal | This question connects the content to the learner’s own experiences and emotions, which is central to the Personal dimension focusing on identity, motivation, and affect. | Clear and meaningful; it encourages personal reflection and emotional connection, fostering deeper engagement with the content. |
| **Cognitive:** What strategies can you use to remember the steps of photosynthesis and the role of chlorophyll in converting light energy into chemical energy? | Cognitive | This question targets metacognitive and cognitive strategies for learning and remembering content, fitting the Cognitive dimension focused on thinking processes and strategies. | Clear and relevant; it promotes strategic thinking and self-regulation, encouraging students to actively manage their learning. |
| **Knowledge-Building:** How does the process of photosynthesis connect to the survival of other living organisms beyond plants, and why is it essential for life on Earth? | Knowledge-Building | This question emphasizes constructing and integrating scientific knowledge, understanding systems and interconnections, which aligns with the Knowledge-Building dimension. | Clear, relevant, and promotes meaningful thinking; it encourages synthesis and application of knowledge to broader ecological contexts. |

---

**Evaluation of Annotations**

| Annotation | Explanation Clarity | Support for Deeper Understanding or Connections |
|------------|---------------------|-------------------------------------------------|
| Photosynthesis: A natural process by which certain organisms convert light energy into chemical energy, enabling them to produce food. | Clear and concise definition of the core concept. | Provides foundational understanding essential for grasping the topic. |
| Green plants, algae, bacteria: Types of organisms capable of photosynthesis; green plants and algae contain chlorophyll, while some bacteria have similar pigments. | Clear explanation distinguishing different photosynthetic organisms. | Supports understanding of biological diversity and complexity of photosynthesis. |
| Light energy: Energy from sunlight that powers the photosynthesis process. | Clear and straightforward. | Connects energy source to the process, aiding comprehension of energy flow. |
| Chemical energy: Energy stored in chemical bonds, such as those in glucose molecules produced during photosynthesis. | Clear explanation linking chemical energy to glucose. | Helps students understand energy transformation and storage. |
| Glucose: A simple sugar that serves as an energy source and building block for growth in plants and other photosynthetic organisms. | Clear and informative. | Connects molecular product to plant growth and energy use, deepening understanding. |
| Carbon dioxide: A gas absorbed from the air, used as a carbon source to build glucose molecules. | Clear and precise. | Clarifies the role of CO2 in photosynthesis, supporting comprehension of inputs. |
| Water: Absorbed by roots or cell membranes, water provides electrons and hydrogen atoms needed for glucose synthesis. | Clear and detailed. | Explains the chemical role of water, enhancing understanding of the process. |
| Chlorophyll: A green pigment in chloroplasts that captures sunlight and initiates the energy conversion. | Clear and focused explanation. | Highlights the functional importance of chlorophyll, linking structure to function. |
| Sunlight: The natural light from the sun, essential for driving photosynthesis. | Simple and clear. | Reinforces the energy source, supporting conceptual clarity. |
| Oxygen: A byproduct released into the atmosphere during photosynthesis, vital for respiration in most living organisms. | Clear and relevant. | Connects photosynthesis to ecological and biological significance. |
| Biological process: A series of chemical reactions occurring within living organisms to sustain life. | Clear general definition. | Helps situate photosynthesis within broader biological context. |
| Energy conversion: The transformation of energy from one form (light) to another (chemical) during photosynthesis. | Clear and precise. | Clarifies a key scientific concept underlying photosynthesis. |
| Plant growth: The increase in plant size and biomass supported by the glucose produced through photosynthesis. | Clear and relevant. | Links photosynthesis to tangible biological outcomes, supporting application. |
| Overall, photosynthesis is fundamental for life on Earth as it produces food and oxygen, supporting ecosystems and the planet’s energy flow. | Clear summary statement. | Synthesizes key ideas, reinforcing the importance and interconnectedness of the process. |

---

**Summary**

- The comprehension questions align well with their respective RA dimensions, each promoting relevant and meaningful thinking that supports the framework’s goals.
- All questions are clear, relevant, and encourage reflection, strategy use, social interaction, or knowledge integration.
- The annotations provide clear, concise explanations of key terms and concepts, effectively supporting deeper understanding and useful connections.
- Together, the questions and annotations form a coherent set that supports multiple dimensions of the Reading Apprenticeship framework.

### ChatGPT-4o Result

Step 1: Extracted Info
Photosynthesis is a crucial biological process in which green plants, algae, and some bacteria convert light energy into chemical energy stored in glucose. Key components of this process include:

- Light energy (usually from the sun)
- Chlorophyll (green pigment in plant cells)
- Carbon dioxide absorption from the air
- Water absorption from the soil
- Conversion of raw materials into glucose
- Release of oxygen as a byproduct

Photosynthesis supports plant growth and energy needs and produces oxygen necessary for most living organisms on Earth. Keywords: photosynthesis, light energy, chlorophyll, glucose, carbon dioxide, water, oxygen, plants, algae, bacteria. 

Step 2: Generated Questions
1. **Social**: How do you think the process of photosynthesis might differ in various environments, such as a rainforest versus a desert? Discuss with a partner how environmental factors could influence the efficiency of photosynthesis.

2. **Personal**: Reflect on a time when you spent time in nature, perhaps in a garden or park. How does understanding the process of photosynthesis change your perception of the plants you observed during that experience?

3. **Cognitive**: What strategies can you use to remember the key components and steps of photosynthesis? Consider creating a mnemonic or drawing a diagram to help visualize the process.

4. **Knowledge-Building**: How does the process of photosynthesis contribute to the balance of oxygen and carbon dioxide in the Earth's atmosphere? Explain the significance of this balance for both plants and animals. 

Step 3: Generated Annotations
**Photosynthesis**: A vital process for life on Earth, where organisms like green plants, algae, and some bacteria convert light energy into chemical energy. This process is essential for producing the food and oxygen that sustain most life forms.

**Light energy**: Energy from the sun that is captured by plants and other photosynthetic organisms. This energy initiates the photosynthesis process, driving the conversion of carbon dioxide and water into glucose.

**Chlorophyll**: The green pigment found in the chloroplasts of plant cells. Chlorophyll absorbs sunlight, which is crucial for converting light energy into chemical energy during photosynthesis.

**Carbon dioxide absorption**: Plants take in carbon dioxide (CO2) from the air through small openings in their leaves called stomata. This gas is a key raw material needed for photosynthesis.

**Water absorption**: Plants absorb water from the soil through their roots. Water is another essential raw material for photosynthesis, providing hydrogen atoms that are incorporated into glucose.

**Conversion into glucose**: The primary goal of photosynthesis is to convert light energy into glucose, a simple sugar that serves as an energy source for plants and other organisms that consume them.

**Release of oxygen**: As a byproduct of photosynthesis, oxygen is released into the atmosphere. This oxygen is critical for the survival of most living organisms, as it is used in cellular respiration to release energy from food.

**Plant growth and energy needs**: Photosynthesis not only fuels plant growth by providing energy in the form of glucose but also supports the broader ecosystem by producing oxygen and serving as the foundation of the food chain.

**Keywords**: These terms are central to understanding photosynthesis and its role in the ecosystem, highlighting the interconnectedness of energy conversion, plant biology, and the sustenance of life on Earth. 

Step 4: Quality Evaluation
**Questions Evaluation:**

1. **Social**: 
   - **Alignment**: This question aligns with the Social dimension as it encourages discussion and collaboration with a partner, focusing on how environmental factors affect photosynthesis.
   - **Justification**: By discussing with a partner, students engage in social learning, sharing insights and perspectives.
   - **Evaluation**: The question is clear and relevant, promoting meaningful thinking about environmental influences on biological processes.

2. **Personal**: 
   - **Alignment**: This question aligns with the Personal dimension as it asks students to reflect on their own experiences and connect them to the scientific concept of photosynthesis.
   - **Justification**: It encourages personal reflection and connection to prior experiences, enhancing personal engagement with the material.
   - **Evaluation**: The question is clear and relevant, fostering a personal connection to the content that can deepen understanding.

3. **Cognitive**: 
   - **Alignment**: This question aligns with the Cognitive dimension as it focuses on strategies for understanding and remembering the process of photosynthesis.
   - **Justification**: It encourages the use of cognitive strategies like mnemonics and diagrams to aid memory and comprehension.
   - **Evaluation**: The question is clear and relevant, promoting active engagement with the material through cognitive strategies.

4. **Knowledge-Building**: 
   - **Alignment**: This question aligns with the Knowledge-Building dimension as it explores the broader implications of photosynthesis on the Earth's atmosphere.
   - **Justification**: It encourages students to build on their knowledge by understanding the ecological significance of photosynthesis.
   - **Evaluation**: The question is clear and relevant, encouraging students to think about the interconnectedness of biological processes and their ecological impact.

**Annotations Evaluation:**

- **Photosynthesis**: Clearly explains the key idea of photosynthesis and its importance, supporting a deeper understanding of its role in sustaining life.
- **Light energy**: Clearly explains the term and its role in photosynthesis, supporting connections to the process.
- **Chlorophyll**: Clearly explains the term and its function, aiding understanding of its role in photosynthesis.
- **Carbon dioxide absorption**: Clearly explains the process and its importance, supporting understanding of photosynthesis inputs.
- **Water absorption**: Clearly explains the process and its role, aiding comprehension of photosynthesis inputs.
- **Conversion into glucose**: Clearly explains the goal of photosynthesis, supporting understanding of energy conversion.
- **Release of oxygen**: Clearly explains the byproduct of photosynthesis, supporting understanding of its ecological significance.
- **Plant growth and energy needs**: Clearly explains the broader impact of photosynthesis, supporting connections to ecosystem dynamics.
- **Keywords**: Highlights essential terms, supporting understanding of photosynthesis and its ecological role.

Overall, the questions and annotations effectively align with the RA dimensions and promote a comprehensive understanding of photosynthesis through clear explanations and relevant connections.

## Summary

- Geminis are doing better jobs than GPTs in annotation task by generating annotations for both main ideas and keywords, while GPTs overlook the main ideas.
- Gemini-2.5-Flash is a newer model than Gemini-1.5-Pro, and it has better multimodel processing ability to process the texts, and graphs.
- GPT-4.1 and GPT-4.1-mini can provide a better and clear output quality compared to GPT-4o.
- Consider the pricing, we exclude Gemini-2.5-Pro (not available for free trial) and GPT-4.1 (most expensive). When it comes to GPT-4.1-mini and Gemini-2.5-Flash, GPT-4.1-mini is cheaper than Gemini-2.5-Flash without losing too much quality. But I think Gemini-2.5-Flash is more suitable for our project beacuse the free trial Google provides can basically satisfy the needs for testing and demo.

In a summary, I recommend Gemini-2.5-Flash as our LLM.